In [0]:
DATA_FILES = [
    # '/datascope/subaru/data/targeting/MW/S25A_May-June/ga_targets_outerdisk_l90_b28_faint.ecsv',
    # '/datascope/subaru/data/targeting/MW/S25A_May-June/ga_targets_outerdisk_l90_b29_faint.ecsv',

    # '/datascope/subaru/data/targeting/MW/outerdisk_l90_bm28_SSP/ga_targets_outerdisk_l90_bm28_faint.ecsv',
    # '/datascope/subaru/data/targeting/MW/outerdisk_l90_bm29_SSP/ga_targets_outerdisk_l90_bm29_faint.ecsv',

    # '/datascope/subaru/data/targeting/MW/outerdisk_l90_b25_SSP/ga_targets_outerdisk_l90_b25_faint.ecsv',
    # '/datascope/subaru/data/targeting/MW/outerdisk_l90_b27_SSP/ga_targets_outerdisk_l90_b27_faint.ecsv',

    # 2025-11
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b16_SSP/ga_targets_outerdisk_l180_b16_faint.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b17_SSP/ga_targets_outerdisk_l180_b17_faint.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b18_SSP/ga_targets_outerdisk_l180_b18_faint.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b20_SSP/ga_targets_outerdisk_l180_b20_faint.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b21_SSP/ga_targets_outerdisk_l180_b21_faint.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b22_SSP/ga_targets_outerdisk_l180_b22_faint.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b24_SSP/ga_targets_outerdisk_l180_b24_faint.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b25_SSP/ga_targets_outerdisk_l180_b25_faint.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b27_SSP/ga_targets_outerdisk_l180_b27_faint.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b28_SSP/ga_targets_outerdisk_l180_b28_faint.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b29_SSP/ga_targets_outerdisk_l180_b29_faint.csv',
]

FLUXSTD_FILES = [
    # 2025-11
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b16_SSP/l180b16_fluxstd.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b17_SSP/l180b17_fluxstd.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b18_SSP/l180b18_fluxstd.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b20_SSP/l180b20_fluxstd.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b21_SSP/l180b21_fluxstd.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b22_SSP/l180b22_fluxstd.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b24_SSP/l180b24_fluxstd.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b25_SSP/l180b25_fluxstd.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b27_SSP/l180b27_fluxstd.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b28_SSP/l180b28_fluxstd.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b29_SSP/l180b29_fluxstd.csv',
]

SKY_FILES = [
    # 2025-11
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b16_SSP/l180b16_sky.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b17_SSP/l180b17_sky.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b18_SSP/l180b18_sky.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b20_SSP/l180b20_sky.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b21_SSP/l180b21_sky.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b22_SSP/l180b22_sky.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b24_SSP/l180b24_sky.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b25_SSP/l180b25_sky.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b27_SSP/l180b27_sky.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b28_SSP/l180b28_sky.csv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l180_b29_SSP/l180b29_sky.csv',
]

OBS_TIME = '2025-12-18T09:00:00'  # UTC

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
if 'debug' not in globals():
    import debugpy
    debugpy.listen(('localhost', 5698))
    debug = True

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from pfs.ga.targeting.data import Observation
from pfs.ga.targeting.io import ObservationSerializer

from pfs.ga.targeting.instrument import *
from pfs.ga.common.diagram import FOV, FP, CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.common.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.instrument import SubaruWFC, SubaruPFI

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord

In [0]:
# Load the data files

obs = []
for i, f in enumerate(DATA_FILES):
    s = ObservationSerializer(catalog_name='cat_{i + 1}')
    s.column_map = {
        'objid': 'targetid',
        'ra': 'RA',
        'dec': 'Dec'
    }
    obs.append(s.read(f))

In [0]:
obs[0].data.columns

In [0]:
obs[1].data.columns

In [0]:
obs[0].data[['psf_flux_r', 'r_ps1']]

In [0]:
obs[1].data.input_catalogs.unique()

In [0]:
obs[0].data['priority'].unique()

In [0]:
for i in range(len(obs)):
    plt.plot(*obs[i].get_coords(), '.')

# Load fluxstd and sky files

In [0]:
from pfs.ga.targeting.io import PfsFluxStdReader, PfsSkyReader

In [0]:
fluxstd = []
sky = []
for i, (f, s) in enumerate(zip(FLUXSTD_FILES, SKY_FILES)):
    fs = PfsFluxStdReader().read(f)
    ss = PfsSkyReader().read(s)
    # s.column_map = {
    #     'objid': 'targetid',
    #     'ra': 'RA',
    #     'dec': 'Dec'
    # }
    fluxstd.append(fs)
    sky.append(ss)

### Plot the pointings

In [0]:
# l, b = 90.0, 28.0
# gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
# print(gal.icrs)

# l, b = 90.0, 29.0
# gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
# print(gal.icrs)

# l, b = 90.0, -28.0
# gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
# print(gal.icrs)

# l, b = 90.0, -29.0
# gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
# print(gal.icrs)

# l, b = 90.0, 25.0
# gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
# print(gal.icrs)

# l, b = 90.0, 27.0
# gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
# print(gal.icrs)




l = [180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180]
b = [16, 17, 18, 20, 21, 22, 24, 25, 27, 28, 29]
gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
print(gal.icrs)

# print RA/Dec in HMS / DMS
ra_hms = gal.icrs.ra.to_string(unit=u.hour, sep=':', pad=True)
dec_dms = gal.icrs.dec.to_string(unit=u.degree, sep=':', pad=True, alwayssign=True)

for ra, dec in zip(ra_hms, dec_dms):
    print(f"{ra} {dec}")

In [0]:
# pointing = Pointing(273.5, 60.9, posang=0.0)
# print(pointing.ra, pointing.dec)

# pointing = Pointing(340.5, 26.0, posang=0.0)
# print(pointing.ra, pointing.dec)

# pointing = Pointing(279.5, 60.5, posang=0.0)
# print(pointing.ra, pointing.dec)

pointing = Pointing(110, 38, posang=0.0)
# print(pointing.ra, pointing.dec)

wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(wcs.pointing)
fov = FOV(projection=wcs)

In [0]:
# fov.axes[0].limits = (190, 295)
# fov.axes[1].limits = (29, 25)

In [0]:
pointings = [
    # Pointing(273.525, 60.88, posang=0),
    # Pointing(270.725, 61.00, posang=0),

    # Pointing(340.4, 26.50, posang=0),
    # Pointing(341.215, 25.35, posang=0),

    # Pointing(279.0, 60.45, posang=0),
    # Pointing(276.3, 60.7, posang=0),

    # 2025-11
    Pointing(103, 36.1, posang=0),      # 16
    Pointing(104.6, 36.6, posang=0),    # 17
    Pointing(106.15, 37.1, posang=0),   # 18
    Pointing(107.75 , 37.55, posang=0), # 20

    Pointing(109.4, 38 , posang=0),     # 21
    Pointing(111.05, 38.4, posang=0),   # 22
    Pointing(112.7, 38.8, posang=0),    # 24
    Pointing(114.4, 39.2, posang=0),    # 25
    
    Pointing(116.1, 39.55, posang=0),   # 27
    Pointing(117.8, 39.9, posang=0),    # 28
    Pointing(119.55, 40.15, posang=0)   # 29
]

for p in pointings:
    p.obs_time = OBS_TIME

In [0]:


# obs[0].plot(ax, fov, c='red', size=0.5) #, scalex=False, scaley=False)
# obs[1].plot(ax, fov, c='blue', size=0.5) #, scalex=False, scaley=False)

for i in range(len(obs)):
# for i in [10]:
    f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240, subplot_kw=dict(projection=fov.projection.wcs))

    sky[i].plot(ax, fov, c='lightgray', size=0.5)  # , scalex=False, scaley=False)

    obs[i].plot(ax, fov, c='red', size=1.0) #, scalex=False, scaley=False)
    ra, dec = obs[i].get_coords()
    print(i, ra.mean(), dec.mean())

    fluxstd[i].plot(ax, fov, c='black', size=0.5)  # , scalex=False, scaley=False)

    wcs = WcsProjection(pointings[i], proj='TAN')
    wfc = SubaruWFC(wcs.pointing)
    fov = FOV(projection=wcs)

    pfi = SubaruPFI(SubaruWFC(pointings[i]))
    pfi.plot_focal_plane(ax, fov, corners=True, color='k', lw=2, scalex=False, scaley=False)

    f.show()
    # plt.close(f)

ax.grid()